<h1 style="text-align: center;">Lab 4: Variable Importance & Feature Selection</h1>

**At the end of this practical you should be able to:**
1. Compute variable importance measures using both *filter* and *permutation importance* measures and understand the difference
2. Perform and understand the difference between different feature selection methods (minimally optimal)

The data set for the practical is an **Online News Popularity Data Set** where the task is to predict how many shares an article will get as a binary value ("low" or "high").

## This practical is in two parts. The first part will be done as a demonstration. The second part is for you to do.



# Demo
## Demo Part 1: Creating a function to compare and print our variable importances
Since we want to compare different methods for computing variable importance we are going to want to print them side-by-side ordered by ranking.

While we could write the code each time, it gets a bit repetitive. So we'll write a function that takes a dictionary of feature_importance scores and the list of feature names and prints the table.

*Why a dictionary?* As each list of feature importance scores needs a name, so key = name of method, value = feature importance scores.

**You do not have to understand this function for this practical** - just what it does and how to call it. I.e. read the documentation at the top of the function. That said, you should be able to understand it, it only contains python you know/know how to look up. If you've time once you've finished this practical look back of the function and try and understand it. Ask if you have any questions.

In [18]:
def print_variable_importances( feature_names, dict_in, show_top = 10 ):
  """
  Prints a table of feature importance scores.
  
  Keyword arguments
  feature_names -- list of feature names. Must have the same ordering as the 
                   scores in each instance of list_of_scores (see below)
  dict_in       -- dictionary of the form {method_name:list_of_scores} where:
                   method_name    -- string
                   list_of_scores -- list of scores, ordered in the same order
                                     as the passed feature_names
  show_top      -- number of features to show (default 10, None to print all)
  """


  # Implement the definition of None for show_top
  if show_top is None:
    show_top = len(feature_names)
  
  # Set up lists to hold the titles and score_feature tuples
  # We need a list so that they maintain fixed order as we 
  # iterate over them row-by-row.
  to_print_titles = []
  to_print_scores = []
  
  # Pair each list of scores with a copy of the feature names and sort
  # based on the variable importance score descending
  for k, v in dict_in.items():
    # zip pairs, sorted sorts, reverse orders descending
    feature_names_plus_scores = sorted( zip(v, feature_names) )
    feature_names_plus_scores.reverse()
    to_print_titles.append(k)
    to_print_scores.append(feature_names_plus_scores)
    
    
  # Print the scores
  
  # Create a list of strings to print in each header cell
  line_parts = []
  for j in range(len(to_print_titles)):
    line_parts.append('{:<38}'.format(to_print_titles[j]))
  
  # Print each header cell using a separator ' | ', adding the fixed rank column header 
  print('Rank | ' + ' | '.join( ['{:<38}'.format(x) for x in to_print_titles] ) )
  
  # Print the header underline
  print('---- + ' + ' + '.join( [ '-'*38 ]*len(to_print_titles) ) )
  
  # Print each line
  for i in range(show_top):
    # Create a list of strings to print for each row cell
    line_parts = []
    for j in range(len(to_print_titles)):
      line_parts.append(  '{:<30}: {:.4f}'.format(to_print_scores[j][i][1], to_print_scores[j][i][0]) )
    # Print the row by:
    # (1) joining each row cell using a separator ' | '
    # (2) adding the fixed rank column header 
    print( '{:<4} | '.format(str(i)) + ' | '.join(line_parts) )
      
    
 

## Demo Part 2: Data loading and create our training and test splits
You should be familar with this by now. The data set we will use is has been uploaded to
<br>http://www.cs.nott.ac.uk/~pszgss/teaching/OnlineNewsPopularity.csv <br>
Full details about the dataset can be found at:
<br>https://archive.ics.uci.edu/ml/datasets/online+news+popularity


In [19]:
# Load the data
# Check the column names (i.e. print them). What is wrong? Yes, you will need to fix that.
# HINT: Strings have a .strip() method which returns the string without any leading or training whitespace.
#       I'd advise creating a new (corrected) array of column headings and update data.columns

from pandas import read_csv
data = read_csv('http://www.cs.nott.ac.uk/~pszgss/teaching/OnlineNewsPopularity.csv')

In [20]:
data.head(5)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [21]:
data.columns

Index(['url', ' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
       ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min',
       ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
       ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
       ' self_reference_max_shares', ' self_reference_avg_sharess',
       ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday',
       ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday',
       ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02',
       ' LDA_03', ' LDA_04', ' global_subjectivity',
       ' global_sent

In [22]:
new_columns = []
for col in data.columns:
  new_columns.append( col.strip() )

data.columns = new_columns

In [23]:
len(data)

39644

In [24]:
data.shares

0         593
1         711
2        1500
3        1200
4         505
         ... 
39639    1800
39640    1900
39641    1900
39642    1100
39643    1300
Name: shares, Length: 39644, dtype: int64

We now need to select the input and output features.<br>
**HINT:** Not all input features should be included.

The output feature currently is continuous. Recall that, for this task, we want to predict how many shares an article will get as a binary value ("low" or "high"). 

* High is defined as shares >= 1400
* Low is defined as shares < 1400

In [25]:
# Select the input and output features
# url and timedelta are considered non-predictive in the documentation and 
# therefore should not be used as an input feature
X = data.drop(columns = ['shares', 'timedelta', 'url'])
y = data.shares
y.values[y < 1400] = 0
y.values[y>=1400] =  1

Finally we need to split our data into test and training sets.

We will perform all feature selection based on the training set ONLY. 

The test set will be used to test if our feature selection has helped or hurt.

Use a `test_size=0.33` and a `random_state=42`.

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Demo Part 3: Removing features with low variance
The first thing to do is to check to see if any features (nearly) always have the same value.

This is done via a [`VarianceThreshold`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html) object. The `VarianceThreshold` Object takes one parameter:
* threshold -- the variance threshold below which to remove features

`VarianceThreshold` follows the **fit transform** interface and can be used as a pre-processing step within a pipeline.

**`.fit(...)`:** Learns which features have low variance.<br>
**`.transform(...)`:** Returns a copy of the data removing features with variance below the threshold. 

After calling `.fit(..)` you can use the:<br>
`.get_support()` method to get a boolean mask of which features was selected.

Assuming your VarianceThreshold object is called vt, then you can see the list of features that were selected via:<br>
`X_train.columns[vt.get_support()]`
<br>If you do not understand what that line of code is doing, please ask.

**TASKS:**
1. Using the methods detailed above write the code to print the selected features with a threshold of zero.
2. The function `numpy.invert(...)` will invert a binary mask. Write the code to print the removed features rather than those kept.
3. Are any features removed?
4. What happens if you change the threshold? 
5. What would a good value be?

In [27]:
X.describe()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,...,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,10.398749,546.514731,0.548216,0.996469,0.689175,10.883690,3.293638,4.544143,1.249874,4.548239,...,0.353825,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,0.156064
std,2.114037,471.107508,3.520708,5.231231,3.264816,11.332017,3.855141,8.309434,4.107855,0.844406,...,0.104542,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,0.226294
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000
25%,9.000000,246.000000,0.470870,1.000000,0.625739,4.000000,1.000000,1.000000,0.000000,4.478404,...,0.306244,0.050000,0.600000,-0.328383,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000
50%,10.000000,409.000000,0.539226,1.000000,0.690476,8.000000,3.000000,1.000000,0.000000,4.664082,...,0.358755,0.100000,0.800000,-0.253333,-0.500000,-0.100000,0.150000,0.000000,0.500000,0.000000
75%,12.000000,716.000000,0.608696,1.000000,0.754630,14.000000,4.000000,4.000000,1.000000,4.854839,...,0.411428,0.100000,1.000000,-0.186905,-0.300000,-0.050000,0.500000,0.150000,0.500000,0.250000
max,23.000000,8474.000000,701.000000,1042.000000,650.000000,304.000000,116.000000,128.000000,91.000000,8.041534,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000


In [28]:
# TASK 1
import numpy as np
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=0.0) # 将方差小于等于1.0的特征删除

vt.fit(X)
print(X.columns[vt.get_support()])


Index(['n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'global_rate_negat

In [29]:
# TASK 2
vt.fit(X)
print(X.columns[np.invert(vt.get_support())])

# TASK 3: Answer: 
# no, because the threshold is 0.0

# TASK 4: Answer:
# If we increase the value of threshold gradually, the number of the selected features decreases and the number of the removed features increases.

# TASK 5: Answer:
# First, we increase the value of threshold gradually, and when the number of selected variables keep  unchanged, the threshold would be a good value.

Index([], dtype='object')


## Demo Part 4: Univariate variable importance  (known as filter based methods when used for selection)
Features can be ranked by (typically simple) measures of how they **individually** affect the output variable. There are many measures we can use, [a number of them are implemented in sklearn](http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection).

Univariate feature ranking and selection (all measures) are all implemented in sklearn via the [`GenericUnivariateSelect`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.GenericUnivariateSelect.html) object. 

`GenericUnivariateSelect` follows the **fit transform** interface and can be used as a pre-processing step within a pipeline.

**`GenericUnivariateSelect(...)`:** Constructor. Takes parameters which specify which univariate feature importance measure will be calculated (when calling `.fit(...)`) and the strategy to select the "best" features for creating a reduced feature dataset (when calling `.transform(...)`). See the documentation for the list of avaliable univariate scoring functions.

**`.fit(...)`:** Computes variable importance scores. After calling `.fit(...)` the ranking is avaliable within the object as the attribute `.scores_`<br>

**`.transform(...)`:** Takes the input features and creates a new input features dataset (transforms the original dataset into the new dataset) by selecting a subset of the features as defined by the paramters `mode` and `param`. For instance, if the following parameters were passed when creating the GenericUnivariateSelect object (via the cons)  `mode = 'k_best'` and `param = 10`, `.transform(...)` will return a dataset with only the top 10 input features as ranked.

If we only want to rank we still set up a selector and call `.fit(..)` to compute the rankings, but simply do not end up selecting features based on it (via `.transform(...)`).

After calling `.fit(...)` the scores are available as the attribute: `.scores_`

**TASK:** 
1. First ranking the features via the scoring function: `mutual_info_classif` 
2. Create a new dataset `X_train_b` by selecting only the top 10 features. 
3. Unrelated to (1) & (2) create a Pipeline that incorporates kBest features selection via `to mutual_info_classif` where k = 10.


In [30]:
from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.feature_selection import mutual_info_classif  
#Estimate mutual information for a discrete target variable. (mutual_info_regression)
#Mutual information (MI) [1] between two random variables is a non-negative value, 
#which measures the dependency between the variables. It is equal to zero if 
#and only if two random variables are independent, and higher values mean higher dependency.
us = GenericUnivariateSelect( score_func=mutual_info_classif, mode='k_best', param=10 ) # removes all but the k highest scoring features

us.fit(X,y)F


GenericUnivariateSelect(mode='k_best', param=10,
                        score_func=<function mutual_info_classif at 0x7fd0ea44d040>)

In [31]:
feature_importance_scores = {}
feature_importance_scores['Filter'] = us.scores_

print_variable_importances( X_train.columns, feature_importance_scores )

Rank | Filter                                
---- + --------------------------------------
0    | kw_max_avg                    : 0.0354
1    | LDA_02                        : 0.0349
2    | self_reference_max_shares     : 0.0275
3    | LDA_03                        : 0.0260
4    | LDA_00                        : 0.0258
5    | self_reference_avg_sharess    : 0.0255
6    | LDA_04                        : 0.0250
7    | self_reference_min_shares     : 0.0236
8    | LDA_01                        : 0.0234
9    | kw_avg_avg                    : 0.0224


In [32]:
import pandas as pd
X_new = pd.DataFrame(us.transform(X))
X_new.head(3)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,496.0,496.0,496.0,0.500331,0.378279,0.040005,0.041263,0.040123
1,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001
2,0.0,0.0,918.0,918.0,918.0,0.217792,0.033334,0.033351,0.033334,0.682188


In [33]:
# Demo Task 4.1: First create the GenericUnivariateSelect object, then fit it to compute the scores

from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.feature_selection import mutual_info_classif

us = GenericUnivariateSelect( score_func=mutual_info_classif, mode='k_best', param=10 )

us.fit(X,y)
us.scores_

array([0.00241314, 0.00106773, 0.        , 0.00074268, 0.00142929,
       0.00433493, 0.003496  , 0.00925216, 0.00277746, 0.0006124 ,
       0.        , 0.        , 0.0090751 , 0.00197782, 0.00586922,
       0.00524239, 0.00642788, 0.00404207, 0.01851066, 0.01108851,
       0.01595585, 0.00311623, 0.0145863 , 0.0173789 , 0.03523229,
       0.02252099, 0.0234771 , 0.02065639, 0.02472744, 0.        ,
       0.00373516, 0.00305178, 0.00221219, 0.        , 0.00874051,
       0.00402403, 0.01044491, 0.02535043, 0.02241863, 0.03448697,
       0.0256491 , 0.02484497, 0.00354284, 0.00423748, 0.00692306,
       0.        , 0.00685123, 0.00917724, 0.00451724, 0.00053008,
       0.        , 0.00105561, 0.00313059, 0.0001305 , 0.00110673,
       0.00633661, 0.        , 0.00067176])

In [37]:
# Demo Task 4.2: Create a new dataset X_train_b by selecting only the top 10 features.

X_train_b = us.transform(X)

In [38]:
# Demo Task 4.3: Unrelated to (1) & (2) create a Pipeline that incorporates kBest features selection via to mutual_info_classif where k = 10. 
# Terminate the Pipeline with a KNeighborsClassifier
# Evaluate the performance of the classifier. Think about other pre-processing steps that might also be useful and include them.

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score



steps = [('select',StandardScaler()),
         ('ss', GenericUnivariateSelect( score_func = mutual_info_classif, mode = 'k_best', param = 10)),
         ('model', KNeighborsClassifier())]

knn = Pipeline( steps = steps ) 

knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)

scores = accuracy_score(y_test, y_pred)

print(scores)




0.5880149812734082


In [39]:
# Of course 10 may or may not be the best number to pick. If you feel like it look
# back at the scores and try another value.



## Demo Part 5: Model Based Methods
Rather than using external measures of how features **individually** affect the output feature one can rank features by how much they are used in a trained model. An example of this is Random Forests.

In sklearn, Random Forests have an attribute `feature_importances_` which stores the scores each feature as a list denoting their importance. These scores are computed when the model is fit.

**TASK:**
Train a RandomForestClassifier and print out a list of feature importances

In [40]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train, y_train)


feature_importance_scores['Embeded RF'] = rf.feature_importances_

print_variable_importances( X_train.columns, feature_importance_scores, show_top=None )

Rank | Filter                                 | Embeded RF                            
---- + -------------------------------------- + --------------------------------------
0    | kw_max_avg                    : 0.0354 | kw_avg_avg                    : 0.0437
1    | LDA_02                        : 0.0349 | kw_max_avg                    : 0.0403
2    | self_reference_max_shares     : 0.0275 | LDA_02                        : 0.0328
3    | LDA_03                        : 0.0260 | self_reference_min_shares     : 0.0321
4    | LDA_00                        : 0.0258 | kw_avg_max                    : 0.0302
5    | self_reference_avg_sharess    : 0.0255 | LDA_01                        : 0.0298
6    | LDA_04                        : 0.0250 | kw_avg_min                    : 0.0295
7    | self_reference_min_shares     : 0.0236 | LDA_04                        : 0.0294
8    | LDA_01                        : 0.0234 | self_reference_avg_sharess    : 0.0284
9    | kw_avg_avg                    : 0.02

## Part 6: Model Based Methods: Permutation Importance
Looking at the feature importance scores from the random forest it is hard to know if we should keep the features or not. Moreover, if we wanted to consider feature importance from, say a non-linear SVM, we couldn't. Therefore we will now consider *permutation importance*. 

<br>To do this we will use the [`sklearn.inspection.permutation_importance` package](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html#sklearn.inspection.permutation_importance). 

**NOTE: We want our mean decrease in accuracy to be with respect to generalization accuracy (i.e. on an independent test set, not the training set)**. Therefore we need to compute the permutation scores based on both a training and (independent) test set **or** repeatedly break our data into training and test sets and evaluate via a cross-validation procedure. Note the "data" here to split (either manually or via a cross-validation procedure) is our current training data. The current test data is only for testing our prediction accuracy after we have removed features.

### In this version, we are going to create a separate training and test set (from our current train data) for evaluating the MDA.
This is simpliest way to get a permutation importance and involves **wrapping a fitted model** in a `permutation_importance` object. The signiture of the `permutation_importance` constructor is:
`sklearn.inspection.permutation_importance(estimator, X, y, *, scoring=None, n_repeats=5, n_jobs=None, random_state=None, sample_weight=None, max_samples=1.0)`
<br>
The permutation importance of a feature is calculated as follows. First, a baseline metric, defined by scoring, is evaluated on a (potentially different) dataset defined by the X. Next, a feature column from the validation set is permuted and the metric is evaluated again. The permutation importance is defined to be the difference between the baseline metric and metric from permutating the feature column.
<br>**TASK:**
1. Wrap a trained (on X_train, y_train) RandomForest model in a `permutation_importance` object to generate the feature importances. You should pass the X_train and y_train data, with `n_repeats=2, random_state=0`.  
2. Add the feature importances to the results dictionary and print the dictionary to compare the feature importances to those from the previous exercise. These can be accessed from the `.importances_mean` attribute in the `permutation_importance` object.


In [41]:
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
# TASK 1 
rf_perm = RandomForestClassifier()
rf_perm.fit(X_train, y_train)
perm = permutation_importance(rf_perm,X_train, y_train,n_repeats=2, random_state=0 )

In [42]:
# TASK 2
method_name = 'Perm RF ({:.2f})'.format(scores*100)

feature_importance_scores[method_name] = perm.importances_mean

print_variable_importances(X_train.columns, feature_importance_scores)

Rank | Filter                                 | Embeded RF                             | Perm RF (58.80)                       
---- + -------------------------------------- + -------------------------------------- + --------------------------------------
0    | kw_max_avg                    : 0.0354 | kw_avg_avg                    : 0.0437 | kw_avg_avg                    : 0.0135
1    | LDA_02                        : 0.0349 | kw_max_avg                    : 0.0403 | kw_max_avg                    : 0.0117
2    | self_reference_max_shares     : 0.0275 | LDA_02                        : 0.0328 | is_weekend                    : 0.0117
3    | LDA_03                        : 0.0260 | self_reference_min_shares     : 0.0321 | self_reference_min_shares     : 0.0104
4    | LDA_00                        : 0.0258 | kw_avg_max                    : 0.0302 | kw_min_avg                    : 0.0064
5    | self_reference_avg_sharess    : 0.0255 | LDA_01                        : 0.0298 | self_reference_

Looking at the results what do you notice? Specifically consider the permutation importance scores remembering their interpretation. Try summing the features, what do you notice? 
What can we infer from this?

In [17]:
# The prediction score using both two methods are similar.
# Eight of ten features are included in both two methods.
# These eight features should be selected.

## Part 7: Wrapper Methods
We know that considering the effect of holding out variables and/or their relative use within a given model does not show how *predictive* they actually are. This is because other variables may be *masking* and other interaction effects going on.<br>

**Example:** If predicting someones country of birth the feature `country_of_drivers_licence` may not be listed as that predictive as it should have been if the feature `passport_country` exists since the model could have used either in a model dependent way. In random forests they are used equally meaning the predictive amount will be half of what it should have been on average in this case).

Wrapper methods perform feature subset search to try and find an optimal subset of features. Typically this is done for **feature selection** rather than **feature understanding**. 

In the former we are interested in finding a fixed (or minimal) subset of features with the most predictive power. In the latter we are interested in asking questions regarding specific features (or groups of features) and how they individually and in combination with other features affect prediction accuracy. Here we are only going to consider the former. The latter currently must be done manually by holding out variables, writing your own permutation code or using R. See the lecture for more details.


### Part 7.1: Recursive feature elimination (RFE)
In sklearn the RFE Object wraps a model. The model must produce `.feature_importances_` as part of the `.fit(...)` method. 

You've been given the documentation as part of practical for all other methods - this time you will need to go and look at the documentation yourself.

**TASKS:**
1. Create an RFE object (model) with and embedded Random Forest Classifier. Select 10 features.
2. Print the generated feature importances for the selected features. BONUS: Add a list of all features to the results dictionary with zeros when the feature wasn't selected.

In [41]:
# TASK 1
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

selector = RFE(RandomForestClassifier(), n_features_to_select=10)

In [64]:
# TASK 2
selector.fit(X_train, y_train)
print(selector.n_features_)


method_name = 'RFE RF '
feature_importance_scores = {}
feature_importance_scores[method_name] = selector.estimator_.feature_importances_

print_variable_importances( X_train.columns[selector.support_], feature_importance_scores )

10
Rank | RFE RF                                
---- + --------------------------------------
0    | kw_avg_avg                    : 0.1206
1    | kw_max_avg                    : 0.1121
2    | kw_avg_max                    : 0.0982
3    | LDA_01                        : 0.0977
4    | n_unique_tokens               : 0.0974
5    | kw_avg_min                    : 0.0963
6    | LDA_02                        : 0.0956
7    | LDA_04                        : 0.0952
8    | global_sentiment_polarity     : 0.0938
9    | global_subjectivity           : 0.0932


In [ ]:
# BONUS

size = X_train.columns[np.invert(selector.support_)].shape
zeros = np.zeros(shape=size)
feature_scores = {}
feature_scores['RFE RF'] = np.append(selector.estimator_.feature_importances_, zeros)
columns = np.append(X_train.columns[selector.support_], X_train.columns[np.invert(selector.support_)])
print_variable_importances( columns, feature_scores, show_top=None)

Rank | RFE RF                                
---- + --------------------------------------
0    | kw_avg_avg                    : 0.1251
1    | kw_max_avg                    : 0.1104
2    | LDA_01                        : 0.0978
3    | kw_avg_max                    : 0.0977
4    | kw_avg_min                    : 0.0961
5    | n_non_stop_unique_tokens      : 0.0960
6    | global_sentiment_polarity     : 0.0954
7    | LDA_04                        : 0.0947
8    | LDA_02                        : 0.0943
9    | n_tokens_content              : 0.0927
10   | weekday_is_wednesday          : 0.0000
11   | weekday_is_tuesday            : 0.0000
12   | weekday_is_thursday           : 0.0000
13   | weekday_is_sunday             : 0.0000
14   | weekday_is_saturday           : 0.0000
15   | weekday_is_monday             : 0.0000
16   | weekday_is_friday             : 0.0000
17   | title_subjectivity            : 0.0000
18   | title_sentiment_polarity      : 0.0000
19   | self_reference_min_shares  

### Part 7.2: All relevant feature selection
Instead of asking for a fixed number of features, we may simply want to throw away those that are not relevant. This is a more advanced method to **removing features with no/low variance**. I.e. we are looking to throw away features we think do not have any **real** impact. By real impact we mean no significant impact in a generalized sense. While we could have simply defined a threshold for RFE (stopping throwing away features once the permutation importance, i.e. change in mean prediction accuracy, was not zero) we would expect some change in prediction accuracy due to random chance. Addressing this issue the Boruta package finds iteratively throws aways the worst feature of the set, stopping when the change in mean prediction accuracy is greater than chance.

**Note: The approach is still a heuristic and may throw away valuable features since it removes features in a greedy fashion. In addition throwing away features now that look like they have no affect may be premature, as your training set grows they may be important.**

This is done by wrapping a model (again, one whose `.fit(..)` method produces `.feature_importances_`).

**TASKS:**
1. Fit a Boruta Object (model) by wrapping a RandomForestClassifier.
2. Print the selected features.

In [35]:
!pip install -q Boruta # install the package. If this doesn't work run in a terminal: sudo pip3 install Boruta

In [36]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

selector = BorutaPy(RandomForestClassifier())

In [ ]:
selector.fit(X.values, y.values)
features = selector.transform(X.values)
print(features)